<a href="https://colab.research.google.com/github/lmencisoe/CDA/blob/main/Taller3/Modelo_Taller3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pandas-profiling

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pandas_profiling import ProfileReport
import tensorflow as tf
from sklearn.feature_extraction import DictVectorizer

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Input, Rescaling, Lambda, MaxPooling2D, Conv2D, AveragePooling2D
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection  import GridSearchCV
import seaborn as sns
import warnings
import re
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Taller3_CDA/Data_Taller_3_modelo.csv",sep=";")

In [ ]:
df['Attrition'][df['Attrition'] == 'Yes'] = 1
df['Attrition'][df['Attrition'] == 'No'] = 0
df['Attrition'] = df['Attrition'].apply(int)

In [ ]:
df['OverTime'][df['OverTime'] == 'Yes'] = 1
df['OverTime'][df['OverTime'] == 'No'] = 0
df['OverTime'] = df['OverTime'].apply(int)

In [ ]:
df.info()

In [ ]:
X_total = df.drop(['JobSatisfaction'], axis=1)
Y_total = df['JobSatisfaction']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_total, Y_total, test_size=0.2, random_state=2022)

In [ ]:
#multi:softprob

In [ ]:
xgb_model = xgb.XGBClassifier()

parameters = {'nthread':[1,2,3], #when use hyperthread, xgboost may become slower
              'objective':['multi:softprob'],
              'learning_rate': [0.01,0.05,0.1], #so called `eta` value
              'max_depth': [2,4,6],
              'subsample': [0.05,0.1,0.15],
              'colsample_bytree': [0.05,0.1,0.15],
              #'n_estimators': [30,50,100], #number of trees, change it to 1000 for better results
              'seed': [2022]}


xgb_grid = GridSearchCV(xgb_model,
                        parameters,
                        cv = 5,
                        n_jobs = 5,
                        verbose=True)

In [ ]:
with tf.device('/device:GPU:0'): 
    xgb_grid.fit(X_train, 
                 y_train,
                 early_stopping_rounds=20,
                 eval_set = [(X_train, y_train),(X_test, y_test)])    

In [ ]:
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

In [ ]:
xgb_selecc = xgb.XGBClassifier(nthread=5, #when use hyperthread, xgboost may become slower
         objective='multi:softprob',
         learning_rate= 0.01, #so called `eta` value
         max_depth= 6,
         subsample= 0.1,
         colsample_bytree= 0.15,
         seed = 2022)

xgb_selecc.fit(X_train,
         y_train,  
         eval_metric="merror",
         early_stopping_rounds=20,
         eval_set = [(X_train, y_train),(X_test, y_test)])

In [ ]:
pred_modelo_train = xgb_selecc.predict(X_train)
pred_modelo_test = xgb_selecc.predict(X_test)

In [ ]:
class_names = ['1','2','3','4']

In [ ]:
print(classification_report(y_train,pred_modelo_train,target_names=list(class_names)))

In [ ]:
print(classification_report(y_test,pred_modelo_test,target_names=list(class_names)))

In [ ]:
accuracy_score(y_train, pred_modelo_train), accuracy_score(y_test, pred_modelo_test)

#*Random Forest*

In [ ]:
parameters = {
    'n_estimators': [10,20,30],
    'max_depth': [2,5,7],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [3,5,7]
}
regr = RandomForestClassifier(random_state=0)

clf = GridSearchCV(regr, 
                   parameters,
                   cv = 5)
clf.fit(X_train, y_train)

In [ ]:
print(clf.best_score_)
print(clf.best_params_)

In [ ]:
rf_selecc = RandomForestClassifier(
         n_estimators= 20, #so called `eta` value
         max_features = 'log2',
         min_samples_split = 7)

rf_selecc.fit(X_train,
         y_train)

In [ ]:
pred_modelo_train = rf_selecc.predict(X_train)
pred_modelo_test = rf_selecc.predict(X_test)

In [ ]:
print(classification_report(y_train,pred_modelo_train,target_names=list(class_names)))

In [ ]:
print(classification_report(y_test,pred_modelo_test,target_names=list(class_names)))

In [ ]:
accuracy_score(y_train, pred_modelo_train), accuracy_score(y_test, pred_modelo_test)

#*Arbol de decisión*

In [ ]:
parameters = {
    'max_depth': [2,5,7],
    'min_samples_split': [3,5,7],
    'max_features': ['auto', 'sqrt', 'log2']
}
regr_arb = DecisionTreeClassifier(random_state=0)

arbol = GridSearchCV(regr_arb, 
                   parameters,
                   cv = 5)
arbol.fit(X_train, y_train)

In [ ]:
print(arbol.best_score_)
print(arbol.best_params_)

In [ ]:
arbol_selecc = DecisionTreeClassifier(
         max_depth= 2, #so called `eta` value
         min_samples_split = 3,
         max_features = 'log2')

arbol_selecc.fit(X_train,
         y_train)

In [ ]:
pred_modelo_train = arbol_selecc.predict(X_train)
pred_modelo_test = arbol_selecc.predict(X_test)

In [ ]:
print(classification_report(y_train,pred_modelo_train,target_names=list(class_names)))

In [ ]:
print(classification_report(y_test,pred_modelo_test,target_names=list(class_names)))

In [ ]:
accuracy_score(y_train, pred_modelo_train), accuracy_score(y_test, pred_modelo_test)

#*Red Neuronal*

In [ ]:
y_dict = [{'':str(x)} for x in y_train.tolist()]
y_dict_test = [{'':str(x)} for x in y_test.tolist()]


In [ ]:
vec = DictVectorizer(sparse=False, dtype=int)
M_train = vec.fit_transform(y_dict)
M_test = vec.fit_transform(y_dict_test)

In [ ]:
model = Sequential(name="RedNeuronal")
model.add(Input(shape=(44),name='Capa_Entrada'))
model.add(Dense(20,activation='relu',name='Capa_Densa_1'))
model.add(Dense(10,activation='relu',name='Capa_Densa_2'))
model.add(Dense(5,activation='relu',name='Capa_Densa_3'))
model.add(Dense(len(class_names),activation='softmax',name='Capa_Respuesta'))
model.compile(loss='categorical_crossentropy', optimizer='sgd',metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.summary()

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3)
]

In [ ]:
history = model.fit(X_train, 
         M_train,           
        validation_data=([X_test, M_test]),
        epochs=50,
        callbacks=my_callbacks
    )

In [ ]:
pred_train = model.predict(X_train)
y_pred = list()
for i in range(len(pred_train)):
    y_pred.append((pred_train[i] == pred_train[i].max()).astype(int))

In [ ]:
y_pred = np.array(y_pred)
print(classification_report(M_train,y_pred,target_names=list(class_names)))

In [ ]:
accuracy_score(M_train, y_pred)

In [ ]:
pred_test = model.predict(X_test)
y_pred_test = list()
for i in range(len(pred_test)):
    y_pred_test.append((pred_test[i] == pred_test[i].max()).astype(int))

In [ ]:
y_pred_test = np.array(y_pred_test)
print(classification_report(M_test,y_pred_test,target_names=list(class_names)))

In [ ]:
accuracy_score(M_test, y_pred_test)